In [28]:
import openai
import langchain_community
import pinecone 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone,InMemoryVectorStore
# from openai import OpenAI
import yaml
import os
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
# from langchain.vectorstores import Pinecone as PineconeIndex
from langchain.embeddings.openai import OpenAIEmbeddings

In [4]:
os.getcwd()

'c:\\Users\\Programming.com\\Desktop\\MY_project\\langchain_pdf_QA\\langchain_pdf_QA'

In [5]:

with open('config.yaml','r') as f:
    config = yaml.safe_load(f)

openai_api_key = config['OPENAI']['api_key']
model = config['OPENAI']['embade_model']
pinecone_api_key = config['pinecone']['api_key']
pinecone_env =  config['pinecone']['env']
directory = r"C:\Users\Programming.com\Desktop\MY_project\langchain_pdf_QA\langchain_pdf_QA\inputs"

In [86]:
llm = OpenAI(api_key=openai_api_key,temperature=0.9)

In [6]:
def read_doc(directory):
    file_load = PyPDFDirectoryLoader(directory)
    document = file_load.load()
    return document

documt = read_doc(directory)
documt

[Document(metadata={'source': 'C:\\Users\\Programming.com\\Desktop\\MY_project\\langchain_pdf_QA\\langchain_pdf_QA\\inputs\\test_indianeconomy_2024.pdf', 'page': 0}, page_content='ECONOMIC SURVEY 2023 -24 \n \nPRESS INFORMATION BUREAU  \nGOVERNMENT OF INDIA  \n******  \nINDIA’S REAL GDP PROJECTED TO GROW BETWEEN 6.5 –7 PER \nCENT IN 2024 -25 \nSHOWING RESILIENCE, INDIA’S REAL GDP GREW BY 8.2 PERCENT \nIN FY 24, EXCEEDING 8 PERCENT MARK IN THREE OUT OF FO UR \nQUARTERS OF FY 24  \n \n  SHARE S OF  AGRICULTURE, INDUSTRY AND SERVICES SECTOR S \nIN OVERALL GVA AT CURRENT PRICES IN FY 24 WERE 17.7 PER \nCENT, 27.6 PER CENT AND 54.7 PER CENT RESPECTIVELY  \n \n MANUFACTURING SECTOR GROWS  BY 9.9 PER CENT IN FY24 ; \nCONSTRUCTION ACTIV ITIES ALSO REGISTER A GROWTH OF 9.9 \nPER CENT  \n \nRETAIL INFLATION DECLINES  TO 5.4 PER CENT IN FY24 AFTER \nAVERAGING AT 6.7 PERCENT IN FY23 \n \nGROSS FIXED CAPITAL FORMATION (GFCF) FROM PRIVA TE NON -\nFINANCIAL CORPORATION’S INCREASES  BY 19.8 PER CENT I

In [56]:
import re

# Sample clean_text function as provided earlier
def clean_text(text):
    # Remove unnecessary whitespace and newlines
    text = re.sub(r'\n+', ' ', text)  # Replace multiple newlines with a space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    
    # Remove special characters (you can adjust this as needed)
    text = re.sub(r'[^\w\s\.\,\%\-\(\)]', '', text)  # Keep letters, numbers, punctuation, etc.
    
    # Normalize spacing again after removal
    text = text.strip()  # Remove leading and trailing whitespace
    return text


input_content = ""
# Iterate over each Document and clean the page_content
for document in documt:

    cleaned_content = clean_text(document.page_content)
    input_content += cleaned_content  # Update the document with cleaned content

# # Now, documents list has cleaned page_content for each Document
# for document in documt:
#     print(document.page_content)  # Print cleaned content to verify
print(input_content)

ECONOMIC SURVEY 2023 -24 PRESS INFORMATION BUREAU GOVERNMENT OF INDIA INDIAS REAL GDP PROJECTED TO GROW BETWEEN 6.5 7 PER CENT IN 2024 -25 SHOWING RESILIENCE, INDIAS REAL GDP GREW BY 8.2 PERCENT IN FY 24, EXCEEDING 8 PERCENT MARK IN THREE OUT OF FO UR QUARTERS OF FY 24 SHARE S OF AGRICULTURE, INDUSTRY AND SERVICES SECTOR S IN OVERALL GVA AT CURRENT PRICES IN FY 24 WERE 17.7 PER CENT, 27.6 PER CENT AND 54.7 PER CENT RESPECTIVELY MANUFACTURING SECTOR GROWS BY 9.9 PER CENT IN FY24 CONSTRUCTION ACTIV ITIES ALSO REGISTER A GROWTH OF 9.9 PER CENT RETAIL INFLATION DECLINES TO 5.4 PER CENT IN FY24 AFTER AVERAGING AT 6.7 PERCENT IN FY23 GROSS FIXED CAPITAL FORMATION (GFCF) FROM PRIVA TE NON - FINANCIAL CORPORATIONS INCREASES BY 19.8 PER CENT IN FY23 , ACTS AS AN IMPORTANT DRIVER OF GROWTH WITH 4.1 LAKH RESIDENTIAL UNITS SOLD IN THE TO P EIGHT CITIES,IN 2023 REAL ESTATE WITNESSES 33 PER CENT Y -O-Y GROWTH , HIGHEST SINCE 2013 FISCAL DEFICIT OF UNION GOVERNMENT DOWN FROM 6.4 PER CENT OF GDP IN FY

[Document(metadata={'source': 'C:\\Users\\Programming.com\\Desktop\\MY_project\\langchain_pdf_QA\\langchain_pdf_QA\\inputs\\test_indianeconomy_2024.pdf', 'page': 0}, page_content='ECONOMIC SURVEY 2023 -24 PRESS INFORMATION BUREAU GOVERNMENT OF INDIA  INDIAS REAL GDP PROJECTED TO GROW BETWEEN 6.5 7 PER CENT IN 2024 -25 SHOWING RESILIENCE, INDIAS REAL GDP GREW BY 8.2 PERCENT IN FY 24, EXCEEDING 8 PERCENT MARK IN THREE OUT OF FO UR QUARTERS OF FY 24 SHARE S OF AGRICULTURE, INDUSTRY AND SERVICES SECTOR S IN OVERALL GVA AT CURRENT PRICES IN FY 24 WERE 17.7 PER CENT, 27.6 PER CENT AND 54.7 PER CENT RESPECTIVELY MANUFACTURING SECTOR GROWS BY 9.9 PER CENT IN FY24  CONSTRUCTION ACTIV ITIES ALSO REGISTER A GROWTH OF 9.9 PER CENT RETAIL INFLATION DECLINES TO 5.4 PER CENT IN FY24 AFTER AVERAGING AT 6.7 PERCENT IN FY23 GROSS FIXED CAPITAL FORMATION (GFCF) FROM PRIVA TE NON - FINANCIAL CORPORATIONS INCREASES BY 19.8 PER CENT IN FY23 , ACTS AS AN IMPORTANT DRIVER OF GROWTH WITH 4.1 LAKH RESIDENTIAL U

In [59]:
def creating_chunks(input_content,chunk_size=500,chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(input_content)
    return chunks

chunks = creating_chunks(input_content)
chunks

['ECONOMIC SURVEY 2023 -24 PRESS INFORMATION BUREAU GOVERNMENT OF INDIA INDIAS REAL GDP PROJECTED TO GROW BETWEEN 6.5 7 PER CENT IN 2024 -25 SHOWING RESILIENCE, INDIAS REAL GDP GREW BY 8.2 PERCENT IN FY 24, EXCEEDING 8 PERCENT MARK IN THREE OUT OF FO UR QUARTERS OF FY 24 SHARE S OF AGRICULTURE, INDUSTRY AND SERVICES SECTOR S IN OVERALL GVA AT CURRENT PRICES IN FY 24 WERE 17.7 PER CENT, 27.6 PER CENT AND 54.7 PER CENT RESPECTIVELY MANUFACTURING SECTOR GROWS BY 9.9 PER CENT IN FY24 CONSTRUCTION',
 'SECTOR GROWS BY 9.9 PER CENT IN FY24 CONSTRUCTION ACTIV ITIES ALSO REGISTER A GROWTH OF 9.9 PER CENT RETAIL INFLATION DECLINES TO 5.4 PER CENT IN FY24 AFTER AVERAGING AT 6.7 PERCENT IN FY23 GROSS FIXED CAPITAL FORMATION (GFCF) FROM PRIVA TE NON - FINANCIAL CORPORATIONS INCREASES BY 19.8 PER CENT IN FY23 , ACTS AS AN IMPORTANT DRIVER OF GROWTH WITH 4.1 LAKH RESIDENTIAL UNITS SOLD IN THE TO P EIGHT CITIES,IN 2023 REAL ESTATE WITNESSES 33 PER CENT Y -O-Y GROWTH , HIGHEST SINCE 2013 FISCAL DEFICIT

In [60]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=openai_api_key,
    model="text-embedding-3-large",

)

In [61]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

In [62]:
import time
# pc = Pinecone(api_key=pinecone_api_key)
index_name = "testpinecone"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)
index

In [64]:
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
os.environ['PINECONE_API_KEY'] = pinecone_api_key

vector_store = PineconeVectorStore.from_texts(chunks, embeddings, index_name=index_name)
# vector_store = Pinecone.from_texts(chunks, embeddings, index_name=index_name)

In [66]:

query = query = "i need over view of indian economy of each page "
docs = vector_store.similarity_search(query,k=3)
print(docs)

[Document(page_content='-23, MAINLY DUE TO RISING PARTICIPATION OF RURAL WOMEN New Delhi, 22ND July 2024 Indias real GDP is projected to grow between 6.5 7 per cent in 2024 -25. The Indian economy recovered swiftly from the pandemic, with its real GDP in FY2 4 being 20 per cent higher than the pre -COVID, FY20 levels. This was stated by the Economic Survey 2023 -24 presented in Parliament today by the Union Minister of Finance and Corporate Affairs Smt Nirmala Sitharaman. The Survey points out that the d omest ic'), Document(page_content='economic growth was surprisingly robust. The Survey states as per the World Economic Outlook (WEO), April 2024 of the International Monetary Fund (IMF) , the global economy registered a growth of 3.2 per cent in 2023 . NBSNCPSMECONOMIC SURVEY 2023 -24 PRESS INFORMATION BUREAU GOVERNMENT OF INDIA ECONOMIC SURVEY CONSERVATIVELY PROJECTS A REAL GDP GROWTH OF 6.5 7 PER CENT IN FY25 REAL GDP GROWS BY 8.2 PER CENT IN FY24 EXCEEDS 8 PER CENT IN THREE OUT OF 

In [110]:
embedding_vector = embeddings.embed_query(query)
print(embedding_vector)
docs = vector_store.similarity_search_by_vector_with_score(embedding_vector,k=2)
# print(docs[0].page_content)
for res, score in docs:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

[-0.010141726583242416, 0.005372878164052963, -8.680591236043256e-06, -0.03859821707010269, -0.016331162303686142, -0.014079103246331215, 0.001535241142846644, 0.06091001629829407, -0.05044017732143402, 0.022117912769317627, 0.04381822422146797, 0.00030947179766371846, 0.03740507364273071, -0.06186453253030777, -0.01825510710477829, -0.003458254272118211, 0.002263245638459921, -0.022326713427901268, 0.05276680737733841, 0.011700271628797054, -0.005182720720767975, -0.009522782638669014, 0.002792703453451395, -0.008754696696996689, -0.05825527012348175, 0.021894197911024094, -0.00017372833099216223, -0.011215556412935257, -0.008530981838703156, 0.0029865894466638565, 0.017882250249385834, 0.029515407979488373, 0.013132045045495033, -0.03245352581143379, -0.03537672758102417, 0.0066480510868132114, 0.008680124767124653, -0.04322165250778198, -0.017375163733959198, -0.009127553552389145, -0.010380355641245842, 0.024041857570409775, -0.00233222427777946, 0.03710678964853287, -0.00142897677

In [115]:
chain = load_qa_chain(llm=llm,chain_type="refine") #map_rerank,stuff,refine



In [116]:
query = "i need over view of indian economy of each page "
docs = vector_store.similarity_search(query)
answer = chain.run(input_documents=docs, question=query)
print(answer)


The Economic Survey of 2023-24 highlights the importance of a strong and stable Indian economy that can sustain growth and create opportunities for its citizens. With job creation as a top priority, the survey identifies the need for an average of nearly 78.5 lakh jobs every year in non-farm sectors until 2030. However, in light of the increasingly challenging global environment, the survey also emphasizes the need for domestic efforts to drive economic recovery and growth. Additionally, the tripartite compact proposed by the survey aims to further strengthen the economy and make it more resilient to future disruptions. Overall, while there may be challenges, the Indian economy shows potential for growth and development in the coming years.


In [127]:
local_vector_memory = InMemoryVectorStore(embeddings)
db = local_vector_memory.from_texts(chunks,embeddings)

query = "what is the gdp of 2024 "
docs = db.similarity_search(query)
answer = chain.run(input_documents=docs, question=query)
print(answer)



Based on the Economic Survey and projections from the International Monetary Fund, the GDP of India in FY24 is estimated to be between 6.5 to 7 percent. This is a conservative estimate as India's real GDP grew by 8.2 percent in FY24 and exceeded 8 percent in three out of four quarters. The share of agriculture, industry, and services sectors in the overall GVA at current prices in FY24 were 17.7 percent, 27.6 percent, and 54.7 percent respectively, with the manufacturing sector growing by 9.9 percent. With successful management of administrative and monetary policies, India's retail inflation decreased from 6.7 percent in FY23 to 5.4 percent in FY24. Additionally, the Current Account Deficit (CAD) improved from 2.0 percent of GDP in FY23 to 0.7 percent of GDP in FY24. The Indian economy also recovered and expanded in an orderly fashion post-pandemic, with the real GDP in FY24 being 20 percent higher than its level in FY20. However, the exact GDP for 2024 is not mentioned. 


In [ ]:
The projected GDP for 2024 is between 6.5-7% according to the Economic Survey presented in Parliament by the Union Minister of Finance and Corporate Affairs. This projection is supported by the strong economic growth of 8.2% recorded in FY24 and the expected continued growth in FY25. Additionally, the global economy is also expected to experience a growth of 3.2% in 2023, according to the IMF's World Economic Outlook. 
In FY24, India's real GDP grew by 8.2%, exceeding the 8% mark in three out of four quarters. The share of agriculture, industry, and services sectors in the overall GVA at current prices in FY24 were 17.7%, 27.6%, and 54.7% respectively. The manufacturing sector also saw significant growth of 9.9% in FY24, contributing to the overall GDP growth. 
With deft management of administrative and monetary policies, retail inflation reduced from 6.7% in FY23 to 5.4% in FY24. Additionally, the current account deficit (CAD) improved from 2.0% of GDP in FY23 to 0.7% of GDP in FY24. This shows